# Downstream application package

Invoke the downstream application package that calculates the mean.

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-workflow/downstream/



## Setup

In [15]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.3.0/app-water-bodies.0.3.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the discovery step

Inspect and use `cwltool` to run the CommandLineTool definition:

The CWL document below shows the `stac-client search` step named `discovey`:

In [16]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][0].steps.discovery' -

label: Discovery Step
in:
  api_endpoint: stac_api_endpoint
  search_request: search_request
run: https://github.com/eoap/schemas/releases/download/0.2.0/stac-api-client.0.2.0.cwl
out:
  - search_output


We can extract inspect the CWL with:

In [17]:

cwl_url=$(cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][0].steps.discovery.run' -)
curl -L ${cwl_url} | yq -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2983  100  2983    0     0  11481      0 --:--:-- --:--:-- --:--:-- 11481
cwlVersion: v1.2
class: CommandLineTool
id: stac-client
label: STAC Client Tool
doc: |
  This tool uses the STAC Client to search for STAC items
hints:
  - class: DockerRequirement
    dockerPull: ghcr.io/eoap/schemas/stac-api-client@sha256:a7e346f704836d07f5dabc6b29ee3359e7253f4a294d74f3899973b8920da6f7
requirements:
  - class: InlineJavascriptRequirement
  - class: NetworkAccess
    networkAccess: true
  - class: SchemaDefRequirement
    types:
      - $import: https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml
      - $import: https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml
      - $import: |-
          https://raw.githubusercontent.com

Run the CWL description for the `stac-client search` command line tool wrapped in a CWL CommandLineTool description:

In [25]:
cat <<'EOF' > discovery-params.yaml
bands:
- green
- nir
search_request:
  bbox:
  - -121.399
  - 39.834
  - -120.74
  - 40.472
  collections:
  - sentinel-2-l2a
  datetime_interval:
    end:
      value: '2021-08-01T23:59:59'
    start:
      value: '2021-06-01T00:00:00'
  limit: 20
  max-items: 10
api_endpoint:
  headers: []
  url:
    value: https://earth-search.aws.element84.com/v1/
EOF

cat discovery-params.yaml | yq .

bands:
  - green
  - nir
search_request:
  bbox:
    - -121.399
    - 39.834
    - -120.74
    - 40.472
  collections:
    - sentinel-2-l2a
  datetime_interval:
    end:
      value: '2021-08-01T23:59:59'
    start:
      value: '2021-06-01T00:00:00'
  limit: 20
  max-items: 10
api_endpoint:
  headers: []
  url:
    value: https://earth-search.aws.element84.com/v1/


In [20]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${cwl_url} \
    discovery-params.yaml > discovery-results.json 2> discovery.log

Let's look at the content of the stderr:

In [21]:
cat discovery.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO [job stac-client] /tmp/5qji6z8j$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/5qji6z8j,target=/tdXtcm \
    --mount=type=bind,source=/tmp/hhdgd1r_,target=/tmp \
    --workdir=/tdXtcm \
    --read-only=true \
    --user=1000:1000 \
    --rm \
    --cidfile=/tmp/bned5l_r/20250915142430-967202.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/tdXtcm \
    ghcr.io/eoap/schemas/stac-api-client@sha256:a7e346f704836d07f5dabc6b29ee3359e7253f4a294d74f3899973b8920da6f7 \
    stac-client \
    search \
    https://earth-search.aws.element84.com/v1/ \
    --collections \
    sentinel-2-l2a \
    --bbox \
    -121.399 \
    39.834 \
    -120.74 \
    40.472 \
    --limit \
    20 \
    --max-items \
    10 \
    --datetime \
    2021-06-01T00:00:00/2021-08-01T23:59:59 \
    --save \
    discovery-output.json
INFO [job stac-client] Max memory used: 21MiB
INFO [job stac-client] completed success
INFO Fina

Let's inspect the stdout produced. The `output` block with the id `search_output` of the CWL description is a file:

In [23]:
cat discovery-results.json | jq . -

{
  "search_output": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/discovery-output.json",
    "basename": "discovery-output.json",
    "class": "File",
    "checksum": "sha1$1a8bf9627011fd2da241e6ed5f0d66e75f9a9ed9",
    "size": 229580,
    "path": "/workspace/zarr-cloud-native-format/runs/discovery-output.json"
  }
}


List the discovered STAC Items `id`s:

In [24]:
cat $( cat discovery-results.json | jq -r .search_output.path - ) | jq -r .features[].id -

S2A_10TFK_20210728_0_L2A
S2A_10TFK_20210728_1_L2A
S2B_10TFK_20210723_1_L2A
S2B_10TFK_20210723_0_L2A
S2A_10TFK_20210718_0_L2A
S2B_10TFK_20210713_1_L2A
S2B_10TFK_20210713_0_L2A
S2A_10TFK_20210708_0_L2A
S2A_10TFK_20210708_1_L2A
S2A_10TFK_20210628_0_L2A
